In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [58]:
files = ["jain.txt", "g2-2-100.txt", "g2-2-20.txt", "pathbased.txt", "Aggregation.txt"]

for file in files :
    f_in = open("../src/"+file, "r")
    f_out = open("../src/"+file.replace(".txt", "")+"_out.txt", "w")
    for line in f_in :
        print(re.sub("\s+", ",", line.strip()))
        f_out.write(re.sub("\s+", ",", line.strip())+"\n")
    f_in.close()
    f_out.close()



585,631
607,573
611,590
582,591
571,596
581,597
600,608
593,596
564,602
634,585
626,594
561,581
586,591
617,617
601,611
598,567
598,581
612,639
618,573
556,596
647,604
582,606
626,574
578,604
596,576
634,605
604,576
609,610
632,626
622,577
591,587
611,595
551,573
599,608
628,608
636,614
604,588
595,619
634,569
651,623
616,611
637,578
597,592
640,572
593,635
629,615
584,585
608,590
611,598
585,589
621,615
602,610
612,608
597,618
598,603
583,612
629,593
624,599
566,610
579,588
591,587
648,596
591,593
634,622
630,612
611,636
608,583
586,588
593,618
580,589
590,605
567,587
632,610
614,627
601,622
627,623
555,629
591,606
594,570
586,598
638,643
580,580
593,612
609,640
620,604
633,593
599,613
592,624
586,598
612,594
614,586
542,629
585,607
594,602
610,601
621,537
601,637
602,617
622,633
587,614
578,621
613,622
609,619
573,597
630,598
640,588
614,590
615,579
618,642
609,626
607,607
548,590
581,653
629,599
598,582
624,583
591,600
651,613
591,609
582,598
605,618
609,604
599,563
566,581
556,605
